In [1]:
import re
import string
import time
from tqdm import tqdm
import unicodedata

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver


In [2]:
def rendering(url):
    driver = webdriver.Chrome('/usr/local/bin/chromedriver') # run ChromeDriver
    driver.get(url) # load the web page from the URL
    time.sleep(3) # wait for the web page to load
    render = driver.page_source # get the page source HTML
    driver.quit() # quit ChromeDriver
    return render # return the page source HTML
    

In [3]:
base_url = 'https://d23.com/'

end_urls = ['0-9'] + list(string.ascii_lowercase)


In [4]:
df = pd.DataFrame(columns=['title', 'd23_link'])

for end in tqdm(end_urls):
    search_url = base_url + f'disney-a-to-z/{end}/'
    
    page = rendering(search_url)

    soup = BeautifulSoup(page, 'html.parser')
    
    page_content = soup.find_all('div', class_='a-z-letter-wrapper')
    
    for content in page_content:
        for a in content.select('a'):
            if ('film' in a.text) and (('(' or ')') in a.text) :
                df = df.append({'title': a.text, 'd23_link': a['href'].lstrip(base_url)}, ignore_index=True)
            

100%|██████████| 27/27 [04:36<00:00, 10.23s/it]


In [6]:
translate_dict = dict([(ord(x), ord(y)) for x,y in zip( u"‘’´“”—–-", u"'''\"\"---")]) 


In [7]:
# fix titles with unicode and non-ascii characters
df['title'] = [unicodedata.normalize('NFD', title.translate(translate_dict)) \
               .encode('ascii', 'ignore') \
               .decode("utf-8")\
               .replace('  ', ' ') for title in df['title']]


In [8]:
df

,title,d23_link
0,10 Things I Hate About You (film),a-to-z/10-things-i-hate-about-you-film/
1,101 Dalmatians (film),a-to-z/101-dalmatians-film/
2,101 Dalmatians II: Patch's London Adventure (f...,a-to-z/101-dalmatians-ii-patchs-london-adventu...
3,"101 Problems of Hercules, The (film)",a-to-z/101-problems-of-hercules-the-film/
4,102 Dalmatians (film),a-to-z/102-dalmatians-film/
...,...,...
2105,Zenith (film),a-to-z/zenith-film/
2106,Zokkomon (film),a-to-z/zokkomon-film/
2107,Zootopia (film),a-to-z/zootopia-film/
2108,Zorro the Avenger (film),a-to-z/zorro-the-avenger-film/


In [14]:
def get_d23(i, row):
    
    search_url = base_url + row['d23_link']

    page = rendering(search_url)

    soup = BeautifulSoup(page, 'html.parser')

    page_content = soup.find_all('div', class_='entry-content')

    # get film summary
    summary = []
    for content in page_content:
        for p in (content.select('p')):
            summary.extend(p.text)

    # join summary list, fix summaries with unicode and non-ascii characters
    summary = unicodedata.normalize('NFD', "".join(summary).translate(translate_dict)).encode('ascii', 'ignore').decode("utf-8").replace('  ', ' ')
    
    return summary


In [15]:
df['summary'] = ''


In [16]:
for i,row in tqdm(df.iterrows(), total=df.shape[0]):

    if row['summary'] == '':
        summary = get_d23(i,row)
    
        df.at[i,'summary'] = summary


100%|██████████| 2110/2110 [6:29:23<00:00, 11.07s/it]  


In [17]:
df

,title,d23_link,summary
0,10 Things I Hate About You (film),a-to-z/10-things-i-hate-about-you-film/,10 Things I Hate About You (film) Bianca and K...
1,101 Dalmatians (film),a-to-z/101-dalmatians-film/,101 Dalmatians (film)Live-action version of th...
2,101 Dalmatians II: Patch's London Adventure (f...,a-to-z/101-dalmatians-ii-patchs-london-adventu...,101 Dalmatians II: Patch's London Adventure (f...
3,"101 Problems of Hercules, The (film)",a-to-z/101-problems-of-hercules-the-film/,"101 Problems of Hercules, The (film) Televisio..."
4,102 Dalmatians (film),a-to-z/102-dalmatians-film/,102 Dalmatians (film) Cruella De Vil is releas...
...,...,...,...
2105,Zenith (film),a-to-z/zenith-film/,Zenith (film) Animated short film; premiered J...
2106,Zokkomon (film),a-to-z/zokkomon-film/,Zokkomon (film) Motion picture produced by Wal...
2107,Zootopia (film),a-to-z/zootopia-film/,Zootopia (film) Animated feature taking place ...
2108,Zorro the Avenger (film),a-to-z/zorro-the-avenger-film/,Zorro the Avenger (film) Foreign theatrical co...


In [28]:
# remove title from beginning of summary
df['summary'] = df.apply(lambda x: x['summary'][x['summary'].find(x['title'])+len(x['title']):].lstrip(), axis=1)

In [42]:
df['summary'].iloc[72] = "19-year-old Alice, trying to escape from an unwanted marriage proposal, falls down a hole and returns to the whimsical world she first encountered as a young girl. She has no memory of her previous visit, and her childhood friends the Mad Hatter, the White Rabbit, Tweedledee and Tweedledum, the March Hare, the Dormouse, the Caterpillar, and the Cheshire Cat wonder if she is the right Alice. She embarks on a fantastical journey through what she is told is really called Underland to find her true destiny, as predicted in a magical scroll, and end the Red Queen's reign of terror. Directed by Tim Burton. Released on March 5, 2010, in regular, 3D and Imax versions, after a February 25, 2010, Royal World Premiere in London. Stars Mia Wasikowska (Alice), Johnny Depp (Mad Hatter), Helena Bonham Carter (Iracebeth, the Red Queen), Anne Hathaway (White Queen), Crispin Glover (The Knave of Hearts), Matt Lucas (Tweedledee and Tweedledum), Leo Bill (Hamish), with the voices of Alan Rickman (Absolem, the Blue Caterpillar), Stephen Fry (Cheshire Cat), Michael Sheen (The White Rabbit), Timothy Spall (Bayard), Christopher Lee (Jabberwocky), Paul Whitehouse (March Hare), Barbara Windsor (Dormouse). 109 min. Based on the books by Lewis Carroll. Tim Burton and Johnny Depp had earlier collaborated on Ed Wood for Disney. Danny Elfman composed the score. The film won two Academy Awards, for Art Direction and for Costume Design."
df['summary'].iloc[72]


"19-year-old Alice, trying to escape from an unwanted marriage proposal, falls down a hole and returns to the whimsical world she first encountered as a young girl. She has no memory of her previous visit, and her childhood friends the Mad Hatter, the White Rabbit, Tweedledee and Tweedledum, the March Hare, the Dormouse, the Caterpillar, and the Cheshire Cat wonder if she is the right Alice. She embarks on a fantastical journey through what she is told is really called Underland to find her true destiny, as predicted in a magical scroll, and end the Red Queen's reign of terror. Directed by Tim Burton. Released on March 5, 2010, in regular, 3D and Imax versions, after a February 25, 2010, Royal World Premiere in London. Stars Mia Wasikowska (Alice), Johnny Depp (Mad Hatter), Helena Bonham Carter (Iracebeth, the Red Queen), Anne Hathaway (White Queen), Crispin Glover (The Knave of Hearts), Matt Lucas (Tweedledee and Tweedledum), Leo Bill (Hamish), with the voices of Alan Rickman (Absolem

In [143]:
df['summary'].iloc[210] = "The beautiful Belle ignores her suitor, the vain Gaston, as she cares for her father, the eccentric Maurice. When Maurice stumbles upon a foreboding castle while lost in the woods, the servants, enchanted into household objects, try to make him welcome, but he is thrown into the dungeon by the Beast. Belle comes to rescue her father and agrees to remain in the castle as his substitute. In order to break the spell, Beast must learn to love another and to be loved in return. Belle seems a likely candidate, but it takes Beast a while to reign in his temper. Belle desperately misses her father, so Beast sadly allows her to leave. Gaston, realizing Beast is a rival for Belle's affection, leads the townsfolk to storm the castle. Belle rushes back in time to profess her love for Beast, and the spell is broken. Initial release in New York on November 13, 1991; general release on November 22, 1991. Directed by Gary Trousdale and Kirk Wise. 84 min. Voices include Paige O'Hara (Belle), Robby Benson (Beast), Richard White (Gaston), Jerry Orbach (Lumiere), David Ogden Stiers (Cogsworth), Angela Lansbury (Mrs. Potts), Jo Ann Worley (Wardrobe). Academy Award nominee in seven categories, including, for the first time for an animated feature, that of Best Picture, it won for Best Song (\"Beauty and the Beast\" by Howard Ashman and Alan Menken) and Best Original Score. Angela Lansbury sang the title song in the story, and Celine Dion and Peabo Bryson sang another rendition over the film's end credits. Since lyricist Howard Ashman had died earlier in the year, the film was dedicated to him: \"To our friend, Howard, who gave a mermaid her voice and a beast his soul, we will be forever grateful.\" Production of the film took three and a half years and required the talents of nearly 600 animators, artists, and technicians. Portions of the film were animated at Disney's satellite facility at Disney-MGM Studios in Lake Buena Vista, Florida. Art directors working on the film travelled to the Loire valley in France for inspiration, and studied the great French romantic painters such as Fragonard and Boucher. It was Ashman who came up with the idea of turning the enchanted objects into living creatures with unique personalities. Glen Keane, the supervising animator on Beast, created his own hybrid beast by combining the mane of a lion, the beard and head structure of a buffalo, the tusks and nose bridge of a wild boar, the heavily muscled brow of a gorilla, the legs and tail of a wolf, and the big and bulky body of a bear. Computer-generated imagery was used in several parts of the film, most notably in the \"Be Our Guest\" sequence and in the creation of a striking three-dimensional ballroom background, allowing dramatic camera moves on the animated characters as they danced. It became the most successful animated feature in motion picture history up to that time, with domestic box office revenues in excess of $140 million. Released on video in 1992. The film was reissued in IMAX and other giant screen theaters on January 1, 2002, featuring the song \"Human Again\" in a never-before-seen animated sequence. New running time 90 min. A 3D version was released on January 13, 2012, on a program with the short Tangled Ever After."
df['summary'].iloc[210]


'The beautiful Belle ignores her suitor, the vain Gaston, as she cares for her father, the eccentric Maurice. When Maurice stumbles upon a foreboding castle while lost in the woods, the servants, enchanted into household objects, try to make him welcome, but he is thrown into the dungeon by the Beast. Belle comes to rescue her father and agrees to remain in the castle as his substitute. In order to break the spell, Beast must learn to love another and to be loved in return. Belle seems a likely candidate, but it takes Beast a while to reign in his temper. Belle desperately misses her father, so Beast sadly allows her to leave. Gaston, realizing Beast is a rival for Belle\'s affection, leads the townsfolk to storm the castle. Belle rushes back in time to profess her love for Beast, and the spell is broken. Initial release in New York on November 13, 1991; general release on November 22, 1991. Directed by Gary Trousdale and Kirk Wise. 84 min. Voices include Paige O\'Hara (Belle), Robby B

In [46]:
df['summary'].iloc[211] = 'A live-action telling of the classic story. A prince is turned into a hideous beast by a spell that can only be broken by true love. The beautiful Belle is trapped in his castle, discovering not only the Beast but also that the staff has been turned into household objects. The Beast realizes that this human girl, the first to visit the castle since it was enchanted, may be his last hope. Directed by Bill Condon. Released Mar. 17, 2017, after a Mar. 15 release in the Philippines. Stars Emma Watson (Belle), Luke Evans (Gaston), Dan Stevens (Beast), Ewan McGregor (Lumiere), Emma Thompson (Mrs. Potts), Josh Gad (LeFou), Ian McKellen (Cogsworth), Kevin Kline (Maurice), Audra McDonald (Garderobe), Stanley Tucci (Cadenza). 129 min. The Beast is almost all CG throughout the movie, a process which is also used with great effect in a spectacular "Be Our Guest" number. All of the songs have been retained from the 1991 animated feature, with three more added-"Evermore," "Days in the Sun," and "How Does a Moment Last Forever"-written by the original composer, Alan Menken, with lyrics by Sir Tim Rice. Filmed in widescreen format at Shepperton Studios outside London. It was nominated for Academy Awards for Best Costume Design and Best Production Design.'
df['summary'].iloc[211]


'A live-action telling of the classic story. A prince is turned into a hideous beast by a spell that can only be broken by true love. The beautiful Belle is trapped in his castle, discovering not only the Beast but also that the staff has been turned into household objects. The Beast realizes that this human girl, the first to visit the castle since it was enchanted, may be his last hope. Directed by Bill Condon. Released Mar. 17, 2017, after a Mar. 15 release in the Philippines. Stars Emma Watson (Belle), Luke Evans (Gaston), Dan Stevens (Beast), Ewan McGregor (Lumiere), Emma Thompson (Mrs. Potts), Josh Gad (LeFou), Ian McKellen (Cogsworth), Kevin Kline (Maurice), Audra McDonald (Garderobe), Stanley Tucci (Cadenza). 129 min. The Beast is almost all CG throughout the movie, a process which is also used with great effect in a spectacular "Be Our Guest" number. All of the songs have been retained from the 1991 animated feature, with three more added-"Evermore," "Days in the Sun," and "Ho

In [49]:
df['summary'].iloc[366] = 'Computer-animated feature that presents a new twist to the classic fable of a young chicken who causes widespread panic when he mistakes a falling acorn for a piece of the sky. Chicken Little is determined to restore his reputation, but just as things are starting to go his way, a real piece of the sky lands on his head. Chicken Little and his band of misfit friends--Abby Mallard (aka Ugly Duckling), Runt of the Litter, and Fish Out of Water--attempt to save the world without sending the town into a whole new panic. Released on Nov. 4, 2005. Directed by Mark Dindal. Voices include Zach Braff (Chicken Little), Patrick Stewart (Mr. Woolensworth), Joan Cusack (Abby Mallard), Steve Zahn (Runt of the Litter), Amy Sedaris (Foxy Loxy), Don Knotts (Mayor Turkey Lurkey), Garry Marshall (Buck Cluck), Wallace Shawn (Principal Fetchit), Dan Molina (Fish out of Water). 81 min. The film opened in 100 theaters in a new Disney digital 3-D process. Released on DVD in 2006.'
df['summary'].iloc[366]


'Computer-animated feature that presents a new twist to the classic fable of a young chicken who causes widespread panic when he mistakes a falling acorn for a piece of the sky. Chicken Little is determined to restore his reputation, but just as things are starting to go his way, a real piece of the sky lands on his head. Chicken Little and his band of misfit friends--Abby Mallard (aka Ugly Duckling), Runt of the Litter, and Fish Out of Water--attempt to save the world without sending the town into a whole new panic. Released on Nov. 4, 2005. Directed by Mark Dindal. Voices include Zach Braff (Chicken Little), Patrick Stewart (Mr. Woolensworth), Joan Cusack (Abby Mallard), Steve Zahn (Runt of the Litter), Amy Sedaris (Foxy Loxy), Don Knotts (Mayor Turkey Lurkey), Garry Marshall (Buck Cluck), Wallace Shawn (Principal Fetchit), Dan Molina (Fish out of Water). 81 min. The film opened in 100 theaters in a new Disney digital 3-D process. Released on DVD in 2006.'

In [52]:
df['summary'].iloc[387] = "Live-action retelling of the Cinderella story. Young Ella's merchant father remarries after the death of her mother. Eager to support her loving father, Ella welcomes her new stepmother and her daughters Anastasia and Drisella into the family home. But, when Ella's father unexpectedly dies, she finds herself at the mercy of a jealous and cruel new family. Finally relegated to nothing more than a servant girl covered in ashes, and spitefully named Cinderella, Ella could easily begin to lose hope. But Ella does not give in to despair, and meets a dashing stranger in the woods. Unaware that he is really a prince, Ella finally feels she has met a kindred soul. It appears her fortunes may be about to change when the palace sends out an open invitation for all maidens to attend a ball. Alas, her stepmother forbids her to attend. But help is at hand when a kindly beggar woman steps forward and, with a pumpkin and a few mice, changes Cinderella's life forever. Released in the U.S. on March 13, 2015, after numerous international releases on March 6, 11, and 12. Directed by Kenneth Branagh. Stars Lily James (Ella), Cate Blanchett (Lady Tremaine), Holliday Grainger (Anastasia), Sophie McShera (Drisella), Richard Madden (Kit/Prince Charming), Helena Bonham Carter (Fairy Godmother), Stellan Skarsgard (Grand Duke), Derek Jacobi (King). 105 min. Filmed in widescreen format."
df['summary'].iloc[387]


"Live-action retelling of the Cinderella story. Young Ella's merchant father remarries after the death of her mother. Eager to support her loving father, Ella welcomes her new stepmother and her daughters Anastasia and Drisella into the family home. But, when Ella's father unexpectedly dies, she finds herself at the mercy of a jealous and cruel new family. Finally relegated to nothing more than a servant girl covered in ashes, and spitefully named Cinderella, Ella could easily begin to lose hope. But Ella does not give in to despair, and meets a dashing stranger in the woods. Unaware that he is really a prince, Ella finally feels she has met a kindred soul. It appears her fortunes may be about to change when the palace sends out an open invitation for all maidens to attend a ball. Alas, her stepmother forbids her to attend. But help is at hand when a kindly beggar woman steps forward and, with a pumpkin and a few mice, changes Cinderella's life forever. Released in the U.S. on March 13

In [54]:
df['summary'].iloc[585] = 'A live-action telling of the classic story. Circus owner Max Medici enlists former circus star Holt Farrier and his children, Milly and Joe, to care for a newborn elephant whose oversized ears make him a laughing stock in an already struggling circus. But when they discover that Dumbo can fly, the circus makes an incredible comeback, attracting persuasive entrepreneur V.A. Vandevere, who recruits the peculiar pachyderm for his newest, larger-than-life entertainment venture, Dreamland. Dumbo soars to new heights alongside a charming and spectacular aerial artist Colette Marchant, until Holt learns that, beneath its shiny veneer, Dreamland is full of dark secrets. Directed by Tim Burton. Released Mar. 29, 2019, also in 3D and IMAX, after a Mar. 27 release in Belgium, South Korea, and other countries. Stars Colin Farrell (Holt Farrier), Michael Keaton (V.A. Vandevere), Danny DeVito (Max Medici), Eva Green (Colette Marchant), Nico Parker (Milly Farrier), Finley Hobbins (Joe Farrier). To add a sense of authenticity to the circus, filmmakers recruited fourth-generation circus performer Kristian Kristof from Hungary to share his extensive knowledge and international connections, resulting in a multicultural array of jugglers, clowns, knife throwers, and contortionists. For Dreamland\'s parade sequence, costume designer Colleen Atwood\'s team created more than 200 costumes for the performers, plus an additional 500 for the crowd characters. Sharon Rooney (Miss Atlantis) sings the iconic ballad "Baby Mine," written originally for the 1941 animated film. 112 min. Filmed in widescreen format.'
df['summary'].iloc[585]


'A live-action telling of the classic story. Circus owner Max Medici enlists former circus star Holt Farrier and his children, Milly and Joe, to care for a newborn elephant whose oversized ears make him a laughing stock in an already struggling circus. But when they discover that Dumbo can fly, the circus makes an incredible comeback, attracting persuasive entrepreneur V.A. Vandevere, who recruits the peculiar pachyderm for his newest, larger-than-life entertainment venture, Dreamland. Dumbo soars to new heights alongside a charming and spectacular aerial artist Colette Marchant, until Holt learns that, beneath its shiny veneer, Dreamland is full of dark secrets. Directed by Tim Burton. Released Mar. 29, 2019, also in 3D and IMAX, after a Mar. 27 release in Belgium, South Korea, and other countries. Stars Colin Farrell (Holt Farrier), Michael Keaton (V.A. Vandevere), Danny DeVito (Max Medici), Eva Green (Colette Marchant), Nico Parker (Milly Farrier), Finley Hobbins (Joe Farrier). To a

In [56]:
df['summary'].iloc[860] = 'Fairy tale combining live action and CGI released by Walt Disney Studios Motion Pictures Germany. With the help of a magical book and a clumsy dragon named Hector, young Lilly must prove herself to become the successor of Surulunda the witch. Released Feb. 19, 2009, in Germany, Switzerland, and Austria. Directed by Stefan Ruzowitzky. Stars Alina Freund (Lilly), Anja Kling (Mother), Ingo Naujoks (Hieronymus), Pilar Bardem, (Surulunda), Michael Mittermeier (Hector, voice). 89 min. Produced by Dor Film, Blue Eyes Fiction, Classic Srl, and Buena Vista International Film Production Germany. Based on the internationally best-selling book series Hexe Lilli by Knister.'
df['summary'].iloc[860]


'Fairy tale combining live action and CGI released by Walt Disney Studios Motion Pictures Germany. With the help of a magical book and a clumsy dragon named Hector, young Lilly must prove herself to become the successor of Surulunda the witch. Released Feb. 19, 2009, in Germany, Switzerland, and Austria. Directed by Stefan Ruzowitzky. Stars Alina Freund (Lilly), Anja Kling (Mother), Ingo Naujoks (Hieronymus), Pilar Bardem, (Surulunda), Michael Mittermeier (Hector, voice). 89 min. Produced by Dor Film, Blue Eyes Fiction, Classic Srl, and Buena Vista International Film Production Germany. Based on the internationally best-selling book series Hexe Lilli by Knister.'

In [58]:
df['summary'].iloc[861] = "Fairy tale combining live action and CGI released by Walt Disney Studios Motion Pictures Germany. The throne of Mandolan is bewitched-a big problem for grand vizier Guliman, who wants to be king. His evil sorcerer, Abrash, tries to deceive Lilly into helping them, but there's no fooling a superwitch. With the help of the dragon Hector and a crafty rickshaw driver named Musa, Lilly frees the noble King of Mandolan and his people from the tyranny of Guliman. Released Feb. 17, 2011, in Germany, Switzerland, and Austria. Directed by Harald Sicheritz. Stars Alina Freund (Lilly), Tanay Chheda (Musa), Cosma Shiva Hagen (Suki, voice), Pilar Bardem (Surulunda), Anja Kling (Mother), Michael Mendl (Nandi), Pegah Ferydoni (Leila), Jurgen Tarrach (Guliman). 91 min. Produced by Dor Film, Blue Eyes Fiction, Trixter Productions, Steinweg Emotion Pictures, and Studio Babelsberg, with Buena Vista International Film Production Germany. A sequel to the 2009 film Hexe Lilli: Der Drache und das Magische Buch (Lilly the Witch: The Dragon and the Magic Book)."
df['summary'].iloc[861]


"Fairy tale combining live action and CGI released by Walt Disney Studios Motion Pictures Germany. The throne of Mandolan is bewitched-a big problem for grand vizier Guliman, who wants to be king. His evil sorcerer, Abrash, tries to deceive Lilly into helping them, but there's no fooling a superwitch. With the help of the dragon Hector and a crafty rickshaw driver named Musa, Lilly frees the noble King of Mandolan and his people from the tyranny of Guliman. Released Feb. 17, 2011, in Germany, Switzerland, and Austria. Directed by Harald Sicheritz. Stars Alina Freund (Lilly), Tanay Chheda (Musa), Cosma Shiva Hagen (Suki, voice), Pilar Bardem (Surulunda), Anja Kling (Mother), Michael Mendl (Nandi), Pegah Ferydoni (Leila), Jurgen Tarrach (Guliman). 91 min. Produced by Dor Film, Blue Eyes Fiction, Trixter Productions, Steinweg Emotion Pictures, and Studio Babelsberg, with Buena Vista International Film Production Germany. A sequel to the 2009 film Hexe Lilli: Der Drache und das Magische Bu

In [60]:
df['summary'].iloc[1409] = "A remake of the 1961 Hayley Mills film. This time the identical twins are Hallie and Annie, and when they meet for the first time at Camp Walden for Girls in Maine, they conspire to reunite their mom (a wedding gown designer in London) and dad (a vineyard owner in the Napa Valley), who never should have been apart. Directed by Nancy Meyers. Released on July 29, 1998. Stars Dennis Quaid (Nick Parker), Natasha Richardson (Elizabeth James), Lindsay Lohan (Hallie/Annie), Lisa Ann Walter (Chessy), Elaine Hendrix (Meredith Blake). 128 min. Joanna Barnes, who plays Mrs. Vicki Blake, played the role of Vicki in the original film. Eleven-year-old newcomer Lohan hails from Long Island. After filming in London, the production moved to the Staglin Family vineyard near Rutherford in California's Napa Valley. Camp Seely in Crestline, California, doubled for Camp Walden. Special-effects technology has progressed tremendously in the 37 years since the original film, so it was much easier for the cinematographer, Dean Cundey, to accomplish the more than 100 shots combining both girls. Released on video in 1998."
df['summary'].iloc[1409]


"A remake of the 1961 Hayley Mills film. This time the identical twins are Hallie and Annie, and when they meet for the first time at Camp Walden for Girls in Maine, they conspire to reunite their mom (a wedding gown designer in London) and dad (a vineyard owner in the Napa Valley), who never should have been apart. Directed by Nancy Meyers. Released on July 29, 1998. Stars Dennis Quaid (Nick Parker), Natasha Richardson (Elizabeth James), Lindsay Lohan (Hallie/Annie), Lisa Ann Walter (Chessy), Elaine Hendrix (Meredith Blake). 128 min. Joanna Barnes, who plays Mrs. Vicki Blake, played the role of Vicki in the original film. Eleven-year-old newcomer Lohan hails from Long Island. After filming in London, the production moved to the Staglin Family vineyard near Rutherford in California's Napa Valley. Camp Seely in Crestline, California, doubled for Camp Walden. Special-effects technology has progressed tremendously in the 37 years since the original film, so it was much easier for the cine

In [62]:
df['summary'].iloc[1433] = "For years, old wood carver Mr. Meacham has delighted local children with stories of the fierce dragon that resides deep in the woods of the Pacific Northwest. To his daughter, Grace, who works as a forest ranger, these stories are little more than tall tales, until she meets Pete. Pete is a mysterious 10-year-old with no home or family who claims to live in the woods with a giant green dragon named Elliot, who seems remarkably similar to the dragon from Mr. Meacham's stories. With the help of Natalie, an 11-year-old girl whose father, Jack, owns the local lumber mill, Grace sets out to determine where Pete came from, where he belongs, and the truth about this dragon. Directed by David Lowery. Released August 12, 2016 in the U.S., after an Aug. 10 release in Italy. Stars Bryce Dallas Howard (Grace), Karl Urban (Gavin), Robert Redford (Mr. Meacham), Wes Bentley (Jack), Oona Laurence (Natalie), Oakes Fegley (Pete). 102 min. Filmed in widescreen format in New Zealand. A reimagining of the 1977 Disney film of the same title (for which the dragon's name is spelled Elliott)."
df['summary'].iloc[1433]


"For years, old wood carver Mr. Meacham has delighted local children with stories of the fierce dragon that resides deep in the woods of the Pacific Northwest. To his daughter, Grace, who works as a forest ranger, these stories are little more than tall tales, until she meets Pete. Pete is a mysterious 10-year-old with no home or family who claims to live in the woods with a giant green dragon named Elliot, who seems remarkably similar to the dragon from Mr. Meacham's stories. With the help of Natalie, an 11-year-old girl whose father, Jack, owns the local lumber mill, Grace sets out to determine where Pete came from, where he belongs, and the truth about this dragon. Directed by David Lowery. Released August 12, 2016 in the U.S., after an Aug. 10 release in Italy. Stars Bryce Dallas Howard (Grace), Karl Urban (Gavin), Robert Redford (Mr. Meacham), Wes Bentley (Jack), Oona Laurence (Natalie), Oakes Fegley (Pete). 102 min. Filmed in widescreen format in New Zealand. A reimagining of the

In [64]:
df['summary'].iloc[1513] = 'Documentary released by Buena Vista International (France). A story of the very first cry in life, exploring the universal moment of birth through the experiences of 10 women from different cultures across the Earth. Released Oct. 31, 2007, in France, before releases in Belgium, Japan, and other countries. Directed by Gilles de Maistre. 100 min. From Mai Juin Productions, M6 Films, and Wild Bunch, with participation from Disneynature Productions. The births took place Mar. 29, 2006, in France, Tanzania, Vietnam, Siberia, Mexico, Brazil, Nigeria, Japan, India, and the U.S.A.'
df['summary'].iloc[1513]


'Documentary released by Buena Vista International (France). A story of the very first cry in life, exploring the universal moment of birth through the experiences of 10 women from different cultures across the Earth. Released Oct. 31, 2007, in France, before releases in Belgium, Japan, and other countries. Directed by Gilles de Maistre. 100 min. From Mai Juin Productions, M6 Films, and Wild Bunch, with participation from Disneynature Productions. The births took place Mar. 29, 2006, in France, Tanzania, Vietnam, Siberia, Mexico, Brazil, Nigeria, Japan, India, and the U.S.A.'

In [66]:
df['summary'].iloc[1876] = 'International feature from Walt Disney Studios Motion Pictures. After returning home from a world tour, teen pop star Violetta finds herself at a crossroads. She accepts a spontaneous invitation to travel to a beautiful Italian coastal town, where she recovers from heartbreak and transforms into Tini, the woman and artist she was destined to become. The film was localized for markets around the world, premiering Apr. 23, 2015, in the Netherlands, followed by premieres in France, Spain, Italy, and other countries. Directed by Juan Pablo Buscarini. Stars Martina "Tini" Stoessel (Tini), Jorge Blanco (Leon), Mercedes Lambre (Ludmila), Diego Ramos (German), Clara Alonso (Angie), Adrian Salzedo (Caio), Sofia Carson (Melanie). 99 min. Produced by Gloriamundi Producciones. Shot in Spanish on location in Italy. Digitally released Dec. 6, 2016, on Disney Movies Anywhere and other platforms. Based on Violetta, the first original live-action series from Disney Channel Latin America and EMEA (Europe, Middle East, and Africa).'
df['summary'].iloc[1876]


'International feature from Walt Disney Studios Motion Pictures. After returning home from a world tour, teen pop star Violetta finds herself at a crossroads. She accepts a spontaneous invitation to travel to a beautiful Italian coastal town, where she recovers from heartbreak and transforms into Tini, the woman and artist she was destined to become. The film was localized for markets around the world, premiering Apr. 23, 2015, in the Netherlands, followed by premieres in France, Spain, Italy, and other countries. Directed by Juan Pablo Buscarini. Stars Martina "Tini" Stoessel (Tini), Jorge Blanco (Leon), Mercedes Lambre (Ludmila), Diego Ramos (German), Clara Alonso (Angie), Adrian Salzedo (Caio), Sofia Carson (Melanie). 99 min. Produced by Gloriamundi Producciones. Shot in Spanish on location in Italy. Digitally released Dec. 6, 2016, on Disney Movies Anywhere and other platforms. Based on Violetta, the first original live-action series from Disney Channel Latin America and EMEA (Euro

In [76]:
df['summary'].iloc[259] = 'Visual album released digitally Jul. 31, 2020, on Disney+. Inspired by the 2019 album "The Lion King: The Gift," lessons from the Disney film are reimagined for young kings and queens in search of their own crowns. Written, directed, and exec. produced by Beyonce Knowles-Carter. Celebrity appearances include Jay-Z, Lupita Nyong\'o, Naomi Campbell, Pharrell Williams. 85 min. From Parkwood Entertainment. Released by Walt Disney Studios Motion Pictures.'


In [79]:
df['summary'].iloc[338] = 'Stop-motion animated short; released Jan. 8, 2013, on the Frankenweenie Blu-ray release. Victor plays a favorite homemade sci-fi movie in which his faithful dog, Sparky, fends off an alien invasion in space. Directed by Mark Waring. Features the voice of Charlie Tahan (Victor Frankenstein). 3 min.'


In [82]:
df['summary'].iloc[383] = "The young boy who embarked on countless adventures in the Hundred Acre Wood with his band of spirited and lovable stuffed animals has grown up and lost his way. Christopher Robin is stuck in a job where he is overworked, underpaid, and facing an uncertain future. He has a family of his own, but his work has become his life, leaving little time for his wife and daughter, and he has all but forgotten his idyllic childhood spent with a simple-minded, honey-loving stuffed bear and his friends. But when he is reunited with Winnie the Pooh, now tattered and soiled from years of hugs and play, a spark is rekindled, and he is reminded of the endless days of childlike wonder and make-believe that defined his youth. Following an unfortunate mishap with Christopher Robin's briefcase, Pooh and the rest of the gang step out of the forest and into London to return the crucial possessions, because best friends will always be there for you. Released on Aug. 3, 2018, after an Aug. 1 release in France. Directed by Marc Forster. Stars Ewan McGregor (Christopher Robin), Hayley Atwell (Evelyn), Bronte Carmichael (Madeline Robin), with the voices of Jim Cummings (Winnie the Pooh/Tigger), Brad Garrett (Eeyore), Peter Capaldi (Rabbit), Toby Jones (Owl), Sophie Okonedo (Kanga), Nick Mohammed (Piglet). 104 min. Filmed in widescreen format. It was nominated for the Academy Award for Best Visual Effects of 2018 (Christopher Lawrence, Michael Eames, Theo Jones, Chris Corbould)."


In [85]:
df['summary'].iloc[520] = "Documentary chronicling the making of Disneynature's Dolphin Reef; digitally released Apr. 3, 2020, on Disney+. From wave surfing with dolphins in South Africa to dancing with humpback whales in Hawaii, filmmakers go to great lengths to shed new light on the ocean's mysteries. Narrated by Celine Cousteau. Directed and produced by Keith Scholey. 78 min. From Silverback Films and Disneynature."


In [88]:
df['summary'].iloc[527] = "A Disneynature feature. Echo, a young Pacific bottlenose dolphin, seems far more interested in exploring his spectacular coral reef home and its intriguing inhabitants than learning to survive in it. But lessons from his mother, Kumu, and a few close encounters with some of the ocean's greatest predators may encourage Echo to master his vital role in the marine community after all. Directed by Keith Scholey. Digitally released Apr. 3, 2020, on Disney+. Originally released Mar. 28, 2018, in France as Blue. Narrated by Natalie Portman. 77 min."


In [91]:
df['summary'].iloc[599] = 'A Disneynature feature. African elephant Shani and her spirited son Jomo set out on an epic journey with their herd, traveling hundreds of miles across the vast Kalahari Desert to the Zambezi River. Led by their grand matriarch, Gaia, the family faces brutal heat, dwindling resources, and persistent predators as they follow in the footsteps of their ancestors on a quest to reach paradise. Directed by Mark Linfield. Digitally released Apr. 3, 2020, on Disney+. Narrated by Meghan, The Duchess of Sussex. 89 min.'


In [94]:
df['summary'].iloc[637] = "Documentary chronicling the making of Disneynature's Born in China. Filmmakers travel great distances, climb to extreme altitudes, and endure brutal conditions to capture tender moments in the families of pandas, red-crowned cranes, golden monkeys, and chiru. Digitally released on streaming platforms beginning Dec. 27, 2017. Narrated by Maggie Q. Directed by Ben Wallis. 78 min. From Disneynature. See also Ghost of the Mountains."


In [97]:
df['summary'].iloc[756] = "Documentary chronicling the making of Disneynature's Born in China. An international group of filmmakers sets out on a mission to get up close and personal with a family of elusive snow leopards. Digitally released Jun. 30, 2017, on Disney Movies Anywhere. Narrated by Antoine Fuqua. Directed by Ben Wallis. 78 min. From Disneynature. See also Expedition China."


In [100]:
df['summary'].iloc[761] = "David Dunn pursues Kevin Wendell Crumb's superhuman figure of The Beast in a series of escalating encounters, while the shadowy presence of Elijah Price emerges as an orchestrator who holds secrets critical to both men. Released Jan. 18, 2019 in the U.S., after Jan. 16-17 releases in France, Indonesia, and other countries. Directed by M. Night Shyamalan. Stars James McAvoy (Kevin Wendell Crumb/Dennis/The Beast/Hedwig/other roles), Bruce Willis (David Dunn), Samuel L. Jackson (Elijah Price), Anya Taylor-Joy (Casey Cooke), Sarah Paulson (Dr. Ellie Staple). 129 min. A sequel to Unbreakable (2000), from Touchstone Pictures, and Split (2006), from Universal Pictures, it was distributed domestically by Universal and internationally by Buena Vista International. Shot on location in and around Philadelphia. From Blinding Edge Pictures and Blumhouse with participation from The Walt Disney Studios."


In [103]:
df['summary'].iloc[774] = "Animated short; released Mar. 7, 2017, on the Moana Blu-ray release. On Motunui, Maui tries to catch a fish with his magical fishhook, only to be comically foiled by the ocean. Directed by John Musker and Ron Clements. Voices include Dwayne Johnson (Maui), Auli'i Cravalho (Moana). 2 min."


In [106]:
df['summary'].iloc[926] = "Documentary about the life of lyricist Howard Ashman, whose career and vibrant life were cut short by the AIDS epidemic. Directed by Don Hahn. Premiered Apr. 22, 2018, at the Tribeca Film Festival before an Aug. 7, 2020, digital release on Disney+. From Stone Circle Pictures. 94 min. The film's musical score is by Alan Menken, Ashman's songwriting partner."


In [109]:
df['summary'].iloc[957] = "Documentary chronicling the making of Disneynature's Elephant; digitally released Apr. 3, 2020, on Disney+. Filmmakers face extreme weather, inaccessible terrain, and close encounters with predators to shine a light on elephants and their ancient migrations. Narrated by Jeremy Sisto. Directed by Vanessa Berlowitz. 88 min. From Wildstar Films, Silverback Films, and Disneynature."


In [112]:
df['summary'].iloc[1019] = "Live-action film, based on the Rudyard Kipling stories and inspired by the 1967 animated feature. Mowgli, a man-cub, has been raised by a family of wolves, but he finds he is no longer welcome in the jungle when fearsome tiger Shere Khan, who bears the scars of Man, promises to eliminate what he sees as a threat. Urged to abandon the only home he has ever known, Mowgli embarks on a journey of self-discovery, guided by panther-turned-stern mentor Bagheera and the free-spirited bear Baloo. Along the way, Mowgli encounters jungle creatures who do not exactly have his best interests at heart, including Kaa, a python, and the smooth-talking King Louie. Directed by Jon Favreau. Released on April 15, 2016, also in 3D and 3D IMAX. Stars Neel Sethi (Mowgli). Voices include Idris Elba (Shere Khan), Ben Kingsley (Bagheera), Bill Murray (Baloo), Scarlett Johansson (Kaa), Christopher Walken (King Louie), Lupita Nyong'o (Raksha), Giancarlo Esposito (Akela). Live-action is blended with photorealistic CGI animals and environments, using up-to-the-minute technology to immerse audiences in an enchanting and lush world. It won an Academy Award for Best Visual Effects (Robert Legato, Adam Valdez, Andrew R. Jones, Dan Lemmon)."


In [116]:
df['summary'].iloc[1167] = "High school coach Jim White's job-hopping leads him to predominantly Latino McFarland High School, located in an agricultural community in California's farm-rich central valley. Jim knows he has to make this school his last stop-he's out of options, with both his career and his family-and finds himself in a in a diverse, economically-challenged community that feels worlds apart from his previous hometowns. Admittedly, the White family and the students have a lot to learn about each other, but when Coach White notices the boys' exceptional running ability, things begin to change. Soon something beyond their physical gifts impresses White-the power of family relationships, their unwavering commitment to one another, and their incredible work ethic. With grit and determination, Coach White's unlikely band of runners eventually overcomes the odds to become not only a championship cross-country team but an enduring legacy as well. Along the way, Jim and his family realize that they finally found a place to call home and both he and his team achieve their own kind of American dream. Directed by Niki Caro. Released on February 20, 2015. Stars Kevin Costner (Jim White), Maria Bello (Cheryl), Morgan Saylor (Julie), Elsie Fisher (Jamie), Vincent Martella (Brandon), Daniel Moncada (Eddie), Martha Higareda (Lupe), Vanessa Martinez (Maria Marisol), Carlos Pratts (Thomas). Inspired by a 1987 true story. Filmed in California's San Joaquin Valley in widescreen format."


In [119]:
df['summary'].iloc[1270] = 'Live-action retelling of the story of China\'s legendary warrior who risks everything out of love for her family and country. When the Emperor issues a decree that one man per family must serve in the Imperial Army to defend the country from Northern invaders, Hua Mulan, the eldest daughter of an honored warrior, steps in to take the place of her ailing father. Masquerading as a man, Hua Jun, she is tested every step of the way. She must harness her inner-strength in an epic journey that will transform her into an honored warrior and earn her the respect of a grateful nation and a proud father. Originally planned for theatrical release, it was digitally released Sep. 4, 2020, on Disney+, after a Mar. 9 Hollywood premiere at the Dolby Theatre. Directed by Niki Caro. Stars Yifei Liu (Mulan), Donnie Yen (Commander Tung), Tzi Ma (Zhou), Jason Scott Lee (Bori Khan), Yoson An (Honghui), Ron Yuan (Sergeant Qiang), Gong Li (Xianniang), Jet Li (the Emperor). 115 min. In addition to adapting elements from the 1998 animated film, filmmakers took inspiration from the original Chinese "Ballad of Mulan." Filmed in widescreen format in China, with additional shooting in New Zealand.'


In [122]:
df['summary'].iloc[1286] = 'Virtual reality short; premiered with Frozen 2 Nov. 7, 2019, at the Dolby Theatre in Hollywood. As a mother reads her children a bedtime story, the audience is transported to an enchanted forest outside Arendelle where elemental spirits come to life and myths of the past and future are revealed. Directed by Jeff Gipson. 8 min.'


In [125]:
df['summary'].iloc[1370] = 'Animated Frozen short; digitally released Oct. 23, 2020, on Disney+. The previously untold origins of Olaf are revealed as the snowman first comes to life and searches for his identity in the mountains outside Arendelle. Directed by Dan Abraham and Trent Correy. Voices include Josh Gad (Olaf), Idina Menzel (Elsa), Kristen Bell (Anna), Jonathan Groff (Kristoff), Chris Williams (Oaken).'


In [128]:
df['summary'].iloc[1374] = 'Live-action/CGI film adaptation of the popular children\'s book by Katherine Applegate. Ivan is a 400-pound silverback gorilla who shares a communal habitat in a suburban shopping mall with Stella the elephant, Bob the dog, and various other animals. He has few memories of the jungle where he was captured, but the arrival of a baby elephant named Ruby touches something deep within him. Ivan begins to question his life, where he comes from, and where he ultimately wants to be. Originally planned for theatrical release, it digitally premiered Aug. 21, 2020, on Disney+. Directed by Thea Sharrock. 94 min. Stars Bryan Cranston (Mack), Ramon Rodriguez (George), Ariana Greenblatt (Julia), with the voices of Sam Rockwell (Ivan), Angelina Jolie (Stella), Danny DeVito (Bob the dog), Helen Mirren (Snickers the poodle), Brooklynn Prince (Ruby). Filmed in widescreen format at London\'s Pinewood Studios and in Lakeland, Florida. The original song "Free" was written by Diane Warren and performed by Charlie Puth.'


In [131]:
df['summary'].iloc[1425] = "Documentary chronicling the making of Disneynature's Penguins; digitally released Apr. 3, 2020, on Disney+. Cinematographers brave the brutal Antarctic, pushing through frigid temperatures, hurricane-strength winds, and the overwhelming aroma of one million penguins. Narrated by Blair Underwood. Directed by Alastair Fothergill and Jeff Wilson. 78 min. From Silverback Films and Disneynature."


In [134]:
df['summary'].iloc[1646] = 'Hidden camera comedy produced in South Africa. Leon Schuster returns to prank unsuspecting everyday victims and famous faces using hidden cameras. Directed by Gray Hofmeyr. Released Nov. 29, 2013, in South Africa. Stars Leon Schuster (Schuks), Rob van Vuuren (Wayne), Alfred Ntombela (Shorty), Lare Birk (herself). 97 min. From Touchstone Pictures.'


In [137]:
df['summary'].iloc[1761] = "10-min. retelling of the epic Star Wars saga; premiered Dec. 1, 2015, in the ABC Sound Studio at Disney's Hollywood Studios. It closed in 2018 for seasonal operation, superseded in 2020 by the Mickey Shorts Theater. Also screened in the Tomorrowland Theater at Disneyland during Star Wars events, 2015-2020; in Star Wars Launch Bay at Shanghai Disneyland beginning in Jun. 2016; and in the Discoveryland Theatre at Disneyland Paris, 2017-2018."


In [145]:
# remove 'film' and 'live-action' tags from titles
df['title'] = [title[:max([title.find('(film'), title.find('(live')])].rstrip() for title in df['title']]


In [146]:
df

,title,d23_link,summary
0,10 Things I Hate About You,a-to-z/10-things-i-hate-about-you-film/,"Bianca and Kat Stratford are sisters, but ther..."
1,101 Dalmatians,a-to-z/101-dalmatians-film/,Live-action version of the Dodie Smith book an...
2,101 Dalmatians II: Patch's London Adventure,a-to-z/101-dalmatians-ii-patchs-london-adventu...,An animated direct-to-video sequel to One Hund...
3,"101 Problems of Hercules, The",a-to-z/101-problems-of-hercules-the-film/,"Television show; aired on October 16, 1966. Th..."
4,102 Dalmatians,a-to-z/102-dalmatians-film/,Cruella De Vil is released from prison on good...
...,...,...,...
2105,Zenith,a-to-z/zenith-film/,"Animated short film; premiered January 24, 202..."
2106,Zokkomon,a-to-z/zokkomon-film/,Motion picture produced by Walt Disney Co. Ind...
2107,Zootopia,a-to-z/zootopia-film/,Animated feature taking place in a modern mamm...
2108,Zorro the Avenger,a-to-z/zorro-the-avenger-film/,Foreign theatrical compilation of several Zorr...


In [147]:
df.to_csv('movie_summaries.csv')